# 고도, 경사도 데이터 전처리

### 경상북도 데이터 불러오기

In [ ]:
import pandas as pd
df = pd.read_csv('C:/Users/user/Documents/에이콘아카데미/최종/고도,경사도/cleaned_df.csv')
df

### 격자 500*500 으로 변환

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer

# 좌표를 Point 객체로 변환하여 GeoDataFrame 생성 (좌표계: WGS84)
geometry = [Point(xy) for xy in zip(df["longitude"], df["latitude"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# 미터 기반 좌표계로 변환 (예: UTM Zone 52N - EPSG:32652)
gdf = gdf.to_crs(epsg=32652)

# 500m 격자 인덱스 생성
gdf["x500"] = (gdf.geometry.x // 500).astype(int)
gdf["y500"] = (gdf.geometry.y // 500).astype(int)

# 격자 단위 평균값 계산
agg_df = gdf.groupby(["x500", "y500"])[[
    "2021_elevation", "2021_slope", "2022_elevation", "2022_slope"
]].mean().reset_index()

# 격자 중심 좌표 계산
agg_df["x_center"] = agg_df["x500"] * 500 + 250
agg_df["y_center"] = agg_df["y500"] * 500 + 250

# EPSG:4326로 재변환
transformer = Transformer.from_crs("EPSG:32652", "EPSG:4326", always_xy=True)
agg_df["longitude"], agg_df["latitude"] = transformer.transform(
    agg_df["x_center"].values, agg_df["y_center"].values
)

# 결과 저장
agg_df.to_csv("cleaned_df_500m.csv", index=False)

In [ ]:
df1 = pd.read_csv("cleaned_df_500m.csv")
df1.head()

### 컬럼 설명
### x500, y500: 격자 ID
### x_center, y_center: 격자 중심 좌표 (미터 기준)

### longitude, latitude: 격자 중심 위경도

### elevation, slope: 연도별 평균값